# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [4]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
song_files = get_files(filepath = "data/song_data")

In [8]:
song_files[0]
import json

with open(song_files[0]) as _file_:
    json_file = json.load(_file_)

json_file

{'num_songs': 1,
 'artist_id': 'ARD0S291187B9B7BF5',
 'artist_latitude': None,
 'artist_longitude': None,
 'artist_location': 'Ohio',
 'artist_name': 'Rated R',
 'song_id': 'SOMJBYD12A6D4F8557',
 'title': 'Keepin It Real (Skit)',
 'duration': 114.78159,
 'year': 0}

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = []
song_data

[]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

IndexError: list index out of range

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data")

In [12]:
from io import StringIO

with open(log_files[0]) as _file_:
    logs = [json.loads(log_string) for log_string in _file_.read().split("\n")]

    print(len(logs))
    
new_logs = list(filter(lambda x : x['page'] == 'NextSong', logs))
new_logs[0]

366


{'artist': 'Sydney Youngblood',
 'auth': 'Logged In',
 'firstName': 'Jacob',
 'gender': 'M',
 'itemInSession': 53,
 'lastName': 'Klein',
 'length': 238.07955,
 'level': 'paid',
 'location': 'Tampa-St. Petersburg-Clearwater, FL',
 'method': 'PUT',
 'page': 'NextSong',
 'registration': 1540558108796.0,
 'sessionId': 954,
 'song': "Ain't No Sunshine",
 'status': 200,
 'ts': 1543449657796,
 'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"',
 'userId': '73'}

In [13]:
from datetime import datetime, timedelta

utc_time = datetime(1970, 1, 1) + timedelta(milliseconds=new_logs[0]['ts'])
print(utc_time)

2018-11-29 00:00:57.796000


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
utc_time.strftime("%U")

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_data = [
    new_logs[0]['userId'],
    new_logs[0]['firstName'],
    new_logs[0]['lastName'],
    new_logs[0]['gender'],
    new_logs[0]['level']
]
user_data

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
# get songid and artistid from song and artist tables
song_query_data = [new_logs[2]["artist"], new_logs[2]['song'], new_logs[2]['length']]
print(song_query_data)
print(song_select, song_query_data)
cur.execute(song_select, song_query_data)
results = cur.fetchone()

print(results)
'''
# insert songplay record
songplay_data = ()
cur.execute(songplay_table_insert, songplay_data)
conn.commit()
'''
print(cur.execute("SELECT * FROM {}".format("songs")))

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [ ]:
cur.execute("SELECT * FROM songs")